# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
clean_df=pd.read_csv('..\weather_py\output_data\my_city_weather.csv', index_col=0)
clean_df

,City,City_ID,Country,"Date unix, UTC",lon,lat,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Bredasdorp,1015776,ZA,1608135493,20.04,-34.53,69.80,56,0,8.05
1,Ushuaia,3833367,AR,1608135432,-68.30,-54.80,48.20,70,75,6.93
2,New Norfolk,2155415,AU,1608135717,147.06,-42.78,54.00,79,100,1.28
3,George Town,1735106,MY,1608135717,100.34,5.41,78.80,94,40,2.24
4,Hambantota,1244926,LK,1608135717,81.12,6.12,77.00,100,40,4.70
...,...,...,...,...,...,...,...,...,...,...
557,Wanning,1791779,CN,1608135792,110.40,18.80,65.25,85,92,12.75
558,Muravlenko,1540711,RU,1608135792,74.52,63.79,-2.04,93,98,11.10
559,Benjamin Constant,3665016,BR,1608135793,-70.03,-4.38,84.20,74,40,4.70
560,Poum,2138555,NC,1608135504,164.02,-20.23,78.48,73,42,17.54


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# configure gmaps
gmaps.configure(api_key=gkey)
# Store latitude and longitude in locations
locations = clean_df[["lat", "lon"]]
# Fill NaN values and convert to float
humid = clean_df["Humidity (%)"].astype(float)

In [30]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [103]:
perfect_df=clean_df.loc[(clean_df['Cloudiness (%)']<5)&(clean_df['Max Temperature (F)']>70)&
                        (clean_df['Max Temperature (F)']<80)&
                        (clean_df['Wind Speed (mph)']<10)]
perfect_df

,City,City_ID,Country,"Date unix, UTC",lon,lat,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
65,Richards Bay,962367,ZA,1608135727,32.04,-28.78,73.80,74,0,8.10
93,Port Alfred,964432,ZA,1608135731,26.89,-33.59,72.00,79,0,1.99
101,Kruisfontein,986717,ZA,1608135732,24.73,-34.00,75.00,58,0,5.99
132,Pisco,3932145,PE,1608135725,-76.22,-13.70,72.00,77,0,9.17
151,Lázaro Cárdenas,4026082,MX,1608135528,-102.20,17.96,79.30,63,2,0.92
159,Broken Hill,2173911,AU,1608135740,141.43,-31.95,77.00,41,0,8.05
314,Grand Gaube,934479,MU,1608135761,57.66,-20.01,79.00,83,4,5.99
336,Lira,230166,UG,1608135763,32.91,2.23,76.44,31,2,2.46
387,Trincomalee,1226260,LK,1608135770,81.23,8.57,79.93,77,0,5.12
445,Rocha,3440777,UY,1608135777,-54.33,-34.48,78.19,38,3,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [108]:
hotel_df=perfect_df[["City","Country","lon","lat"]].reset_index(drop=True)
h_name=[]
h_address=[]
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword":"resort",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for i in hotel_df.index:
    # get lat, lng from df
    lat = hotel_df["lat"][i]
    lng = hotel_df["lon"][i]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "resort" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request 
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    try:
        print(name_address['results'][0]['name'])
        h_name.append(name_address['results'][0]['name'])
        h_address.append(name_address["results"][0]["vicinity"])
    except (KeyError, IndexError):
        print('Missing field/result... skipping.')
        h_name.append(None)
        h_address.append(None)
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
#    try:
#        cities_pd.loc[index, "Airport Name"] = name_address["results"]["name"]
#        cities_pd.loc[index, "Airport Address"] = name_address["results"]["vicinity"]
#        cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
#    except (KeyError, IndexError):
#        print("Missing field/result... skipping.")

Harbour Lights Resort
Port Alfred Sands
Missing field/result... skipping.
Huaca Los Cachos
La Salud por la Naturaleza- Clinica de Faciales & Spa
Tarrawingee Holiday Units
LUX Grand Gaube Resort & Villas
Lira Resort Lodge
JKAB Beach Resort
Missing field/result... skipping.
Villa Esmeralda
Hotel Vista del Sol Resort
Haritha Hotel Alampur


In [109]:
hotel_df['Hotel Name']=h_name
hotel_df['Hotel Address']=h_address
hotel_df

,City,Country,lon,lat,Hotel Name,Hotel Address
0,Richards Bay,ZA,32.04,-28.78,Harbour Lights Resort,"Richards Bay Central, Richards Bay"
1,Port Alfred,ZA,26.89,-33.59,Port Alfred Sands,Port Alfred
2,Kruisfontein,ZA,24.73,-34.00,None,None
3,Pisco,PE,-76.22,-13.70,Huaca Los Cachos,Pisco
4,Lázaro Cárdenas,MX,-102.20,17.96,La Salud por la Naturaleza- Clinica de Faciale...,"Plaza Zirahuen, Centro, Lázaro Cárdenas"
5,Broken Hill,AU,141.43,-31.95,Tarrawingee Holiday Units,"253 Wills St, Broken Hill"
6,Grand Gaube,MU,57.66,-20.01,LUX Grand Gaube Resort & Villas,"MU, Coastal Road, Grand Gaube"
7,Lira,UG,32.91,2.23,Lira Resort Lodge,Lira
8,Trincomalee,LK,81.23,8.57,JKAB Beach Resort,"NO. 686/11 N Coast Rd, Trincomalee"
9,Rocha,UY,-54.33,-34.48,None,None


In [113]:
hotel_df.dropna(inplace=True)
hotel_df

,City,Country,lon,lat,Hotel Name,Hotel Address
0,Richards Bay,ZA,32.04,-28.78,Harbour Lights Resort,"Richards Bay Central, Richards Bay"
1,Port Alfred,ZA,26.89,-33.59,Port Alfred Sands,Port Alfred
3,Pisco,PE,-76.22,-13.70,Huaca Los Cachos,Pisco
4,Lázaro Cárdenas,MX,-102.20,17.96,La Salud por la Naturaleza- Clinica de Faciale...,"Plaza Zirahuen, Centro, Lázaro Cárdenas"
5,Broken Hill,AU,141.43,-31.95,Tarrawingee Holiday Units,"253 Wills St, Broken Hill"
6,Grand Gaube,MU,57.66,-20.01,LUX Grand Gaube Resort & Villas,"MU, Coastal Road, Grand Gaube"
7,Lira,UG,32.91,2.23,Lira Resort Lodge,Lira
8,Trincomalee,LK,81.23,8.57,JKAB Beach Resort,"NO. 686/11 N Coast Rd, Trincomalee"
10,Acajutla,SV,-89.83,13.59,Villa Esmeralda,"Sonsonate, Sonsonate, Acajutla"
11,Hualmay,PE,-77.61,-11.10,Hotel Vista del Sol Resort,"San Isidro 381, Hualmay"


In [116]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

,lat,lon
0,-28.78,32.04
1,-33.59,26.89
3,-13.70,-76.22
4,17.96,-102.20
5,-31.95,141.43
6,-20.01,57.66
7,2.23,32.91
8,8.57,81.23
10,13.59,-89.83
11,-11.10,-77.61


In [132]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations,label='H',
    info_box_content=hotel_info)

fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

# Display figure
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))